In [28]:
import pandas as pd

In [29]:
df_all = pd.read_excel("../raw/Estimating_Serum.xlsx", sheet_name="All", skiprows=1)
df_unmodified = pd.read_excel("../raw/Estimating_Serum.xlsx", sheet_name="Unmodified", skiprows=2)
df_modified = pd.read_excel("../raw/Estimating_Serum.xlsx", sheet_name="Modified", skiprows=2)

In [30]:
df_all

,Unnamed: 0,Name of Peptide,Sequence,Modifications,Modifications\n(code),Half-life (hours),Half-life (code),Half-Life (code_code),Assay,Assay (code)
0,NaN,NZ2114,Ac-GFGCNGPWNEDDLRCHNHCKSIKGYKGGYCAKGGFVCKCY-NH2,N.A.,0,1,Low stability,2,rat,2
1,NaN,FB006,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,N.A.,0,1.67,Stable,3,rat,2
2,NaN,FB006,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,N.A.,0,11.4,Very high stability,5,monkey,2
3,NaN,hBD1,Ac-DHYNCVSSGGQCLYSACPIFTKIQGTCYRGKAKCCK-NH2,N.A.,0,80 % intact after 48h,Undegradable,6,human serum,1
4,NaN,hBD3,Ac-GIINTLQKYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRR...,N.A.,0,4,High stability,4,human serum,1
...,...,...,...,...,...,...,...,...,...,...
574,NaN,18-9,zXEaAYQkFL,β-amino acid,9,100% intact after 24h,Undegradable,6,human serum,1
575,NaN,18-10,zXEAaYQkFL,β-amino acid,9,100% intact after 24h,Undegradable,6,human serum,1
576,NaN,"CF-[K13(E-Pam), Q34]hPP",CF-APLEPVYPGDNAK(E-pam)PEQMAQYAADLRRYINMLTRQRY...,Lipidation,4,90% intact after 150h,Undegradable,6,human plasma,1
577,NaN,"CF-[K13 (PEG2), Q34]hPP",CF-APLEPVYPGDNAK(PEG2)PEQMAQYAADLRRYINMLTRQRY-NH2,PEGylation,8,40% intact after 150h,Undegradable,6,human plasma,1


In [31]:
# Eliminar filas sin secuencia
df_unmodified = df_unmodified.dropna(subset=["Sequence"])
df_modified = df_modified.dropna(subset=["Sequence"])

In [32]:
# Revisar si las secuencias están presentes en el dataframe "All"
df_modified["in_all"] = df_modified["Sequence"].isin(df_all["Sequence"])
df_unmodified["in_all"] = df_unmodified["Sequence"].isin(df_all["Sequence"])

In [33]:
df_all = df_all[["Sequence", "Half-life (hours)", "Modifications"]].rename(columns={
        "Sequence": "sequence",
        "Modifications": "modifications",
        "Half-life (hours)": "half-life_hours"
    })

In [34]:
df_all

,sequence,half-life_hours,modifications
0,Ac-GFGCNGPWNEDDLRCHNHCKSIKGYKGGYCAKGGFVCKCY-NH2,1,N.A.
1,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,1.67,N.A.
2,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,11.4,N.A.
3,Ac-DHYNCVSSGGQCLYSACPIFTKIQGTCYRGKAKCCK-NH2,80 % intact after 48h,N.A.
4,Ac-GIINTLQKYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRR...,4,N.A.
...,...,...,...
574,zXEaAYQkFL,100% intact after 24h,β-amino acid
575,zXEAaYQkFL,100% intact after 24h,β-amino acid
576,CF-APLEPVYPGDNAK(E-pam)PEQMAQYAADLRRYINMLTRQRY...,90% intact after 150h,Lipidation
577,CF-APLEPVYPGDNAK(PEG2)PEQMAQYAADLRRYINMLTRQRY-NH2,40% intact after 150h,PEGylation


In [35]:
# Agregar columna para filtrar modificaciones
df_all["is_modified"] = df_all["modifications"].apply(lambda x: True if x != "N.A." else False)
df_all["half-life"] = pd.to_numeric(df_all["half-life_hours"], errors="coerce").apply(lambda x: x * 3600 if pd.notnull(x) else x)
df_all = df_all.drop("half-life_hours", axis=1)

In [36]:
df_all

,sequence,modifications,is_modified,half-life
0,Ac-GFGCNGPWNEDDLRCHNHCKSIKGYKGGYCAKGGFVCKCY-NH2,N.A.,False,3600.0
1,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,N.A.,False,6012.0
2,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,N.A.,False,41040.0
3,Ac-DHYNCVSSGGQCLYSACPIFTKIQGTCYRGKAKCCK-NH2,N.A.,False,NaN
4,Ac-GIINTLQKYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRR...,N.A.,False,14400.0
...,...,...,...,...
574,zXEaAYQkFL,β-amino acid,True,NaN
575,zXEAaYQkFL,β-amino acid,True,NaN
576,CF-APLEPVYPGDNAK(E-pam)PEQMAQYAADLRRYINMLTRQRY...,Lipidation,True,NaN
577,CF-APLEPVYPGDNAK(PEG2)PEQMAQYAADLRRYINMLTRQRY-NH2,PEGylation,True,NaN


In [37]:
df_duplicados = df_all[df_all.duplicated(subset='sequence')] 
df_no_duplicados = df_all[~df_all.duplicated(subset='sequence')]

In [38]:
df_duplicados

,sequence,modifications,is_modified,half-life
2,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,N.A.,False,41040.0
9,Ac-GTCRTFGCTCKPLRCKAPRMVRCGETCMG-NH2,N.A.,False,NaN
10,Ac-GTCRTFGCTCKPLRCKAPRMVRCGETCMG-NH2,N.A.,False,NaN
20,Ac-AAQLRRIGDKVNLRQKLLN,N.A.,False,132.0
21,Ac-AAQLRRIGDKVNLRQKLLN,N.A.,False,630.0
...,...,...,...,...
556,Acetyl-CiS1F2Aze3Y4R5CiiaLA(Me)6H7Q8D9L10Ciii,Amino acid substitution/Methylation,True,10080.0
557,Acetyl-CiS1F2Aze3Y4R5CiiaLA(Me)6H7Q8D9L10Ciii,Amino acid substitution/Methylation,True,25200.0
562,Ac-c[CVDINNNC]-NH2,Cyclization,True,NaN
563,Ac-c[CVDINNNC]-NH2,Cyclization,True,NaN


In [39]:
df_no_duplicados

,sequence,modifications,is_modified,half-life
0,Ac-GFGCNGPWNEDDLRCHNHCKSIKGYKGGYCAKGGFVCKCY-NH2,N.A.,False,3600.0
1,Ac-WEEWDREINNYTKLIHELIEESQNQQEKNEQELL-NH2,N.A.,False,6012.0
3,Ac-DHYNCVSSGGQCLYSACPIFTKIQGTCYRGKAKCCK-NH2,N.A.,False,NaN
4,Ac-GIINTLQKYYCRVRGGRCAVLSCLPKEEQIGKCSTRGRKCCRR...,N.A.,False,14400.0
5,Ac-IWIAQELRRIGDEFNAYYARR-NH2,N.A.,False,3900.0
...,...,...,...,...
574,zXEaAYQkFL,β-amino acid,True,NaN
575,zXEAaYQkFL,β-amino acid,True,NaN
576,CF-APLEPVYPGDNAK(E-pam)PEQMAQYAADLRRYINMLTRQRY...,Lipidation,True,NaN
577,CF-APLEPVYPGDNAK(PEG2)PEQMAQYAADLRRYINMLTRQRY-NH2,PEGylation,True,NaN


In [40]:
df_no_duplicados.to_csv("estimating_serum.csv", index=False)